<a href="https://colab.research.google.com/github/ahmadSum1/MIR_Machine_Learning_PARIS_Sebastien/blob/master/MIRs1_ML_Exercice1_Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import make_blobs
import numpy as np

In [ ]:
X, y = make_blobs(n_samples=300, centers=4, n_features=2, random_state=0)

1

In [ ]:
def  calc_Lq(x1 , x2 , q):
  return (np.sum(np.abs(x1-x2)**q))**(1/q)

In [ ]:
X1 = np.array(X[0,:])
X2 = np.array(X[1,:])


In [ ]:
calc_Lq(X1,X2,2)

4.442211434485271

**2**

In [ ]:
def update_assignation(X,U,q):
  A = []
  for i in range(X.shape[0]):
    dist = np.zeros(len(U.shape[1]))
    for j in range(U.shape[0]):
      dist[j] = calc_Lq(X[i,:],U[j,:],q)
    A.append(np.argmin(dist))
  return A

**3**

In [ ]:
def update_centroid(X,a,K):
  d= X.shape[1]
  U = np.zeros(K,d)
  counts = np.zeros(k)
  for i in range(X.shape[0]):
    for j in range(K):
      if a[i]==K[j]:
        U[j,:]+=X[i,:]
        counts[j] +=1
  return U/counts

**4**



In [ ]:
def  compute_Kmeans_loss(X,U,a,q):
  